In [1]:
import sys, os
from matplotlib import pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from matplotlib.animation import FuncAnimation
import glob
import pickle


In [2]:
data_path = "./ReadEvents/*"

all_files = sorted(glob.glob(data_path ))
# print(all_files)

In [3]:
from DoSpaceTrigger_DumpPickleData import ReadFile


Data = []  # List to hold data from all files

LoadPickle = True
PickleName = 'DoSpaceTrigger_Data.pkl'

if LoadPickle:
    if os.path.exists(PickleName):
        print(f"Pickle file {PickleName} exists, loading data from it.")
        with open(PickleName, 'rb') as f:
            Data = pickle.load(f)
        print(f"Loaded data from {PickleName}, total events: {len(Data)}")
    else:
        print(f"Pickle file {PickleName} does not exist, reading from raw files.")
else:
    for i,file in enumerate(all_files):
        print(f"Reading file {i+1}/{len(all_files)}: {file}", end='\r')
        ReadFile(file,Data)
    
    print(f"\nFinished reading files, total events: {len(Data)}")
    with open(PickleName, 'wb') as f:
        pickle.dump(Data, f)


Pickle file DoSpaceTrigger_Data.pkl exists, loading data from it.
Loaded data from DoSpaceTrigger_Data.pkl, total events: 3600


In [ ]:
# Redo the trigger using spatial analysis instead of time box analysis

def threshold_relaxation_coefficient(Guaranteed_Threshold,Neighbour_signals):
    if type(Neighbour_signals) != np.ndarray: Neighbour_signals = np.array(Neighbour_signals)
    N_Neighbours = len(Neighbour_signals)
    # fractional_neighbour_signals = Neighbour_signals/Guaranteed_Threshold


    coefficient = np.exp(-2.5*np.sum(Neighbour_signals)/N_Neighbours)
    return max(0,coefficient)


def ReTriggerEvent(Event):
    # Extract the relevant data from the event

    N_pixels = Event['TotalPixels']

    PixelData = Event['PixelData']
    pix_Theta = PixelData['Theta']
    pix_Phi = PixelData['Phi']
    pix_Trace = PixelData['Trace']
    pix_RecTrigger = PixelData['RecTrigger']

    pix_MyyTrigger = np.zeros_like(pix_RecTrigger)

    trace_length = pix_Trace.shape[1]

    # First Construct the neighbour map
    neighbours_list = np.zeros((N_pixels,N_pixels), dtype=bool)
    for i_pix in range(N_pixels):
        for j_pix in range(N_pixels):
            if i_pix == j_pix: continue
            pix_distance = np.sqrt( (pix_Theta[i_pix]-pix_Theta[j_pix])**2 + (pix_Phi[i_pix]-pix_Phi[j_pix])**2 )
            if pix_distance < 2.0:  # degrees
                neighbours_list[i_pix,j_pix] = True
    

                
    # Now we can find the approximate trigger window from the RecTrigger
    triggered_positions = np.where(pix_RecTrigger)[0]
    if triggered_positions.size > 0:
        min_pos = np.min(triggered_positions[1])
        max_pos = np.max(triggered_positions[1])
    else:
        min_pos = 0
        max_pos = 99999
    min_pos = max(0,min_pos-3)
    max_pos = min(trace_length,max_pos+3)

    Norm_Signal_Array  = pix_Trace / np.std(pix_Trace[:,:200], axis=1, keepdims=True)

    GuaranteedThreshold = 5 # In units of std deviations

    # Now go over each time bin and ReTrigger
    for t in range(min_pos, max_pos+1):
        signals = Norm_Signal_Array[:,t]

        for i_pix in range(N_pixels):
            if signals[i_pix] > GuaranteedThreshold:
                pix_MyyTrigger[i_pix,t] = True
                continue

            neighbour_indices = np.where(neighbours_list[i_pix])[0]
            neighbour_signals = signals[neighbour_indices]
            neighbour_signals = np.clip(neighbour_signals, -GuaranteedThreshold, GuaranteedThreshold)

            relative_threshold = GuaranteedThreshold * threshold_relaxation_coefficient(GuaranteedThreshold, neighbour_signals)
            if signals[i_pix] > relative_threshold:
                pix_MyyTrigger[i_pix,t] = True
                
    
    # Now we must prune the trigger 

    # First we make sure that once triggered, a pixel stays triggered for at least 3 time bins
    for i_pix in range(N_pixels):
        triggered_times = np.where(pix_MyyTrigger[i_pix])[0]
        for t in triggered_times:
            trigger_bins = pix_MyyTrigger[i_pix, max(0,t-2):min(trace_length,t+3)]
            if len(trigger_bins) < 5: continue
            elif (np.sum(trigger_bins[0:3]) == 3) or (np.sum(trigger_bins[1:4]) == 3) or (np.sum(trigger_bins[2:5]) == 3): continue
            else: pix_MyyTrigger[i_pix, t] = False

    # Other Prunning Steps are not added yet

    Event['MyyTrigger'] = pix_MyyTrigger

    

{'Batch': 300000000, 'Shower': 5844, 'EyeID': '5', 'Gen_LogE': 15.4557, 'Gen_CosZen': 0.543468, 'Gen_Xmax': 651.958, 'Gen_SDPPhi': -0.618939, 'Gen_SDPTheta': 2.02862, 'Gen_Chi0': 2.49036, 'Gen_Rp': 1014.91, 'Gen_T0': 29990.6, 'Gen_CoreEyeDist': 1674.31, 'Gen_Primary': 2212, 'EventClass': "'Muon + Noise'", 'TotalPixels': 166, 'PixelData': {'PixelID': array([310, 333, 334, 312, 332, 356, 335, 311, 354, 289, 313, 336, 377,
       355, 288, 358, 357, 314, 290, 376, 378, 379, 401, 381, 382, 316,
       380, 272, 337, 315, 400, 291, 359, 403, 294, 338, 425, 383, 293,
       402, 251, 405, 360, 319, 340, 399, 296, 406, 339, 384, 362, 361,
       427, 252, 404, 341, 271, 226, 428, 318, 274, 295, 292, 225, 119,
       317, 207, 203, 275, 228, 273, 253, 140, 297, 303, 281, 258, 325,
       324, 237, 164, 302,  73, 259, 304, 236, 182, 184,  95,  50, 142,
       249, 280,  74, 327, 326, 181, 116,  10, 260,  52, 238,  54, 282,
        31, 306, 262, 328, 206, 305,  30, 261, 160, 239, 139, 196, 284,
